# This notebook is used to determine the sensitivity matrix from CWFS data for the Auxiliary Telescope
### It first finds the pairs of CWFS images in the EFD
### It then fits the data a reports the zernikes
### Then plots the relationships and fits a slope for the matrix


## This notebook is heavily referenced in tstn-016.lsst.io - Any changes to this file requires review of the technote.

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample

%matplotlib inline

In [ ]:
# Temporarily needed to run at summit
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ncsa_efd') #summit_efd currently offline so need to use copy at NCSA

## Find donut pairs for CWFS analysis
Query for all the `endReadout` events on the timespan of the night.

In [ ]:
t1 = Time("2020-02-19T05:35", format='isot', scale='tai')
t2 = Time("2020-02-19T11:18", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                           ["imageName", "exposureTime", "groupId", "imageType"], t1, t2)

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 
A pair also has the same groupID

In [ ]:
intra_images = []
extra_images = []
intra_times = []
extra_times = []
intra_exptimes = []
extra_exptimes = []

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout)-2:
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    
    #skip known bad files
    if intra == 'AT_O_20200218_000179' and extra == 'AT_O_20200218_000180':
        i+=2
        continue
    
    if ((end_readout['groupId'][i] == end_readout['groupId'][i+1]) and 
        (end_readout['groupId'][i+1] != end_readout['groupId'][i+2]) and
        (end_readout['imageType'][i] == 'ENGTEST')):
        
        print(f"Got a pair: {intra} x {extra}")
        intra_images.append(intra)
        extra_images.append(extra)
        intra_times.append(end_readout.index[i])
        extra_times.append(end_readout.index[i+1])
        intra_exptimes.append(end_readout['exposureTime'][i])
        extra_exptimes.append(end_readout['exposureTime'][i+1])
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

## Find accomapanying metadata from EFD that accomodates each pair

In [ ]:
# Create numpy data structure to hold items of interest
match_data = np.zeros(npairs, dtype=[('intra', '<U35'), 
                                     ('extra', '<U35'), 
                                     ('az', float), 
                                           ('el', float), 
                                           ('rot_pos', float),
                                           ('x', float), 
                                           ('y', float), 
                                           ('z', float), 
                                           ('u', float), 
                                           ('v', float), 
                                           ('w', float), 
                                     ('m1_pressure', float),
                                     ('dz', float)])

In [ ]:
# Populate the data structure from the pairs found above
for i in range(npairs):
    
    # Determine time stamps for searching for metadata
    # include ability to correct for TAI if required, but set to zero for the moment

    t1 = Time(intra_times[i], scale='tai') - TimeDelta(intra_exptimes[i], format='sec', scale='tai') 
    #- TimeDelta(180., format='sec', scale='tai')
    t2 = Time(extra_times[i], scale='tai') - TimeDelta(2., format='sec', scale='tai')

    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth2CalculatedAngle99"], t1, t2)

    hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                           ["reportedPosition0", "reportedPosition1", "reportedPosition2",
                                           "reportedPosition3", "reportedPosition4", "reportedPosition5"], t1 , t2)

    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                                  ["pressure"], t1, t2)
    
    offset = await efd_client.select_time_series("lsst.sal.ATAOS.logevent_hexapodCorrectionCompleted",
                                                 ["hexapod_x", "hexapod_y", "hexapod_z", "hexapod_u", "hexapod_v", "hexapod_w"], t1, t2)
    
    rot_pos = np.mean(rotator['nasmyth2CalculatedAngle99'])
    el = np.mean(azel['elevationCalculatedAngle99'])
    az = np.mean(azel['azimuthCalculatedAngle99'])
    
    # Take median hexapod position over the time interval
    x = hexapod_vals['reportedPosition0'].median()
    y = hexapod_vals['reportedPosition1'].median()
    z = hexapod_vals['reportedPosition2'].median()
    u = hexapod_vals['reportedPosition3'].median()
    v = hexapod_vals['reportedPosition4'].median()
    w = hexapod_vals['reportedPosition5'].median()

    m1 = np.mean(m1_pressure['pressure'])
    dz = np.nan #round((offset['hexapod_z'][-1] - offset['hexapod_z'][-2])/2.,3)
    print(t1,t2,az, el, rot_pos, x, y, z, u, u, w,dz)
    match_data[i] = (f"{intra_images[i]}.fits", f"{extra_images[i]}.fits", az, el, rot_pos, x, y, z, u, u, w, m1, dz)


In [ ]:
# Convert to dataframe
df = pd.DataFrame(match_data)

In [ ]:
#  Save to a csv file for easy access/tracking
filename="20200123_match_sens_matrix_metadata.csv"
df.to_csv(filename)

# Now reduce the data for each pair to get the zernikes from fitting
## For details on the analysis itself see tstn-015 and example notebook on running latiss_align_cwfs script
### Can restart the notebook here if required so long as the datafile exists.

In [ ]:
import sys
import pandas as pd
import asyncio
import logging
import numpy as np
import matplotlib.pyplot as plt
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
import time

In [ ]:
# Requires new pandas version to use, commenting out but leaving here for when new version is used and 
# an exception will be thrown.
# df2 = pd.DataFrame.read_csv("20200123_match.csv")

In [ ]:
df2 = pd.read_csv(filename)
# Need to add all the zernike terms from the fitting that will be done below
df2['zern_defocus_nm'] = np.zeros(len(df2))
df2['zern_xastig_nm'] = np.zeros(len(df2))
df2['zern_yastig_nm'] = np.zeros(len(df2))
df2['zern_xcoma_nm'] = np.zeros(len(df2))
df2['zern_ycoma_nm'] = np.zeros(len(df2))
df2['zern_xtrefoil_nm'] = np.zeros(len(df2))
df2['zern_ytrefoil_nm'] = np.zeros(len(df2))
df2['zern_sphereical_nm'] = np.zeros(len(df2))

In [ ]:
# alignment script needs to have remotes set to False! Otherwise it'll try to command the hexapod!
script = LatissCWFSAlign(index=1, remotes=False)

In [ ]:
# define the location of the butler repo
script.dataPath='/project/shared/auxTel/'

In [ ]:
def get_visitID_from_filename(filename):
    # Expects AT_O_20200218_000167.fits
    # parse out visitID from filename - this is highly annoying
    tmp=filename.split('_')
    prefix=tmp[2] # dayobs without the dashes

    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('.')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    #print(visitID)
    return visitID

In [ ]:
# for logging
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
# Choose a binning factor to use. Default is 2, but can be changed to 1 if desired. Results should be nearly identical.
script.binning=2

In [ ]:
for n in range(len(df2)):
    # see tstn-015 and example notebook on running latiss_align_cwfs script
    script.angle = df2['rot_pos'][n]-df2['el'][n]
    script.intra_visit_id = get_visitID_from_filename(df2['intra'][n])
    script.extra_visit_id = get_visitID_from_filename(df2['extra'][n])

    start_time=time.time()
    await script.run_cwfs()
    end_time=time.time()
    print('WFE fitting for visitIDs {0} and {1} took {2:0.3f} seconds'.format(script.intra_visit_id, script.extra_visit_id,end_time-start_time)) # 56.7s

    # Display fitting results?
    if (False):
        # plot zernikes
        x = np.arange(9)+4
        plt.plot(x, script.algo.zer4UpNm[:9], 'o-', label=f'{script.dz}')
        xlim = plt.xlim()
        plt.plot(np.arange(15), np.zeros(15)+50, 'b--')
        plt.plot(np.arange(15), np.zeros(15)-50, 'b--')
        plt.xlim(xlim)
        plt.ylabel("Zernike coeff (nm)")
        plt.xlabel("Zernike index")
        plt.grid()
        plt.legend()
        
    if (True):
        # plot image and mask
        fig1 = plt.figure(2, figsize=(12,8))
        ax11 = fig1.add_subplot(121)
        ax11.set_title("defocus 0.8 - intra")
        ax11.imshow(script.I1[0].image0)
        ax11.contour(script.algo.pMask) 
        ax12 = fig1.add_subplot(122)
        ax12.set_title("defocus 0.8 - extra")
        ax12.imshow(script.I2[0].image0)
        ax12.contour(script.algo.pMask) 
        
    # Put results into data structure
    # This will throw errors but don't know how to do this properly!
    df2['zern_defocus_nm'][n] = script.algo.zer4UpNm[0]
    df2['zern_xastig_nm'][n] = script.algo.zer4UpNm[1]
    df2['zern_yastig_nm'][n] = script.algo.zer4UpNm[2]
    df2['zern_xcoma_nm'][n] = script.algo.zer4UpNm[3]
    df2['zern_ycoma_nm'][n]= script.algo.zer4UpNm[4]
    df2['zern_xtrefoil_nm'][n] = script.algo.zer4UpNm[5]
    df2['zern_ytrefoil_nm'][n] = script.algo.zer4UpNm[6]
    df2['zern_sphereical_nm'][n] = script.algo.zer4UpNm[7]


In [ ]:
print('done') # not required, but just nice as it tells you when the above box is completed if you hide the output.

In [ ]:
#write data to a new CSV file
filename="20200123_match_sens_matrix_metadata_plus_zerns.csv"
df2.to_csv(filename)

# Fit the data to derive the Sensitivity Matrix Terms

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def parabola(x,b, x0, a):
    return b + a*(x-x0)**2 
def line(x,b, m):
    return b + m*x 
def invparabola(y,b,x0,a):
    return x0+np.sqrt((y-b)/a)
def invline(y,b,m):
    return (y-b)/m

In [ ]:
#df3 = pd.DataFrame.from_csv("20200123_match_zerns.csv")
# use below for pandas 1.0+
df3 = pd.read_csv(filename)

### Plot Y-Coma as a function of Y-hexapod decentering

In [ ]:
inds = np.arange(0,10) 
# cherry-pick the values desired, should certain not be used
inds = [0,1,2,3,5,6,7,8]

xdata=df3['y'][inds]
ydata=df3['zern_ycoma_nm'][inds]

plt.plot(xdata,ydata,'o')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
popt,pcov = curve_fit(line, xdata, ydata)
print('Wavefront Y-Coma as a function of Y-Hexapod displacement fit intercept and slope ',popt)

plt.plot(x,line(x, *popt))
# plt.xlabel('Hexapod displacement in the plane')
plt.ylabel('Zernike Coefficient in nm')
plt.xlabel('Hexapod Y displacement')
plt.title('Wavefront Y-Coma as a function of hexapod Y-displacement')
plt.show()


### Plot Defocus as a function of Y-hexapod decentering

In [ ]:
#Coma as a function of x,y offset (8-23, 31-35)
inds = np.arange(0,10)
inds = [0,1,2,3,5,6,7,8]
#inds = [0,1,2,5,6,7]


xdata=df3['y'][inds]
ydata=df3['zern_defocus_nm'][inds]

plt.plot(xdata,ydata,'o')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
popt,pcov = curve_fit(line, xdata, ydata)

plt.plot(x,line(x, *popt))
# plt.xlabel('Hexapod displacement in the plane')
plt.ylabel('Zernike Coefficient in nm')
plt.title('Wavefront Defocus as a function of Y-hexapod displacement')
plt.show()

print('Wavefront X-Coma as a function of Hexapod displacement intercept and slope',popt)


### Plot X-Coma as a function of Y-hexapod decentering

In [ ]:
inds = np.arange(0,10) 
# cherry-pick the values desired, should certain not be used
inds = [0,1,2,3,5,6,7,8]

xdata=df3['y'][inds]
ydata=df3['zern_xcoma_nm'][inds]

plt.plot(xdata,ydata,'o')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
popt,pcov = curve_fit(line, xdata, ydata)
print('Wavefront X-Coma as a function of Y-Hexapod displacement fit intercept and slope ',popt)

plt.plot(x,line(x, *popt))
# plt.xlabel('Hexapod displacement in the plane')
plt.ylabel('Zernike Coefficient in nm')
plt.xlabel('Hexapod Y displacement')
plt.title('Wavefront X-Coma as a function of hexapod Y-displacement')
plt.show()


In [ ]:
109/4.8